# Projeto 1 - Ciência dos Dados

Nome: Mariana de Camargo Salles Cezar  
Nome: Pedro Paulo Curiati Tiberio

Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import os
from numpy import arange
import numpy as np
import re 

# Para ter melhor print
from IPython.display import display

Importando a base de dados:

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/marianacezar/Documents/proj 1 cdados- 2021


In [3]:
tweets = pd.read_excel('netflix.xlsx')

In [4]:
tweets.dtypes

Treinamento    object
B1              int64
dtype: object

In [5]:
tweets.head(10)

,Treinamento,B1
0,rt @seriesdatvpt: maratonas fresquinhas na #ne...,0
1,@itsdelicatte esperar a netflix botar uns roma...,0
2,rt @netflixbrasil: assista a filmes e séries p...,1
3,e a netflix que caiu 🤡,1
4,"eu não posso ler as palavras “morte”, “assassi...",1
5,@mysteryguitarm @netflix booooa @mysteryguitar...,0
6,rt @limanths: tomara que a personagem da giova...,0
7,@_luisaanogueira saiu da netflix😭😭 mas queria,1
8,rt @girlgroupsbr: seohyun? a maknae do snsd se...,0
9,@milizi @misscupece chama fake art e tem no ne...,0


___
## Classificador automático de sentimento


Descrição do produto e o que considerou como relevante ou não relevante na classificação dos tweets:

O produto escolhido foi Netflix, que é uma das maiores plataformas de streaming de vídeos no mundo. Com isso é possível assistir séries e filmes quando e onde quiser, tanto em tablets, televisões, smartphones e etc.

Para fazer a classificação manual dos tweets foi levado em consideração para ser relevante se o que está escrito expressa, de maneira direta, um sentimento do cliente ao utilizar o serviço.



___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Selecionando apenas a planilha desejada:

In [6]:
filename = 'netflix.xlsx'

In [7]:
teste= pd.read_excel(filename, sheet_name = 'Teste')
teste.head(5)

,Teste,B1
0,@pdrojao amigoooooo tô precisando de uma série...,0
1,"acabei cidade invisível, achei que a netflix i...",1
2,"pra quem gosta de comédia, assistam ""cabra da ...",1
3,@tyongenrose @iucasmoon helpp okay well comece...,0
4,"a mina simplesmente senta, pega um punhal e di...",0


In [8]:
treinamento= pd.read_excel(filename, sheet_name = 'Treinamento')
treinamento.head(5)

,Treinamento,B1
0,rt @seriesdatvpt: maratonas fresquinhas na #ne...,0
1,@itsdelicatte esperar a netflix botar uns roma...,0
2,rt @netflixbrasil: assista a filmes e séries p...,1
3,e a netflix que caiu 🤡,1
4,"eu não posso ler as palavras “morte”, “assassi...",1


Transformando em lista já separando em relevante e irrelevante.

In [9]:
relevante= treinamento.loc[treinamento["B1"]==1,"Treinamento"]
relevante_list = relevante.tolist()


In [10]:
irrelevante= treinamento.loc[treinamento["B1"]==0,"Treinamento"]
irrelevante_list = irrelevante.tolist()


<font color='magenta'> **FUNÇÃO DE LIMPEZA**: </font> 
Adicionando uma função para a limpeza de mensagens removendo: enter, :, ", ', (, ),http, \n e etc. Não removendo os emojis.

In [29]:
def cleanup(text):
    punctuation = '[!-.:?;@/"\,]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '', text_subbed, flags=re.MULTILINE) #Tirar links
    text_final = text_subbed.rstrip('\n') #Tirar \n
    return text_final
    

Para aplicar a **função limpeza** é necessário que a lista seja transformada em string. Para isso será realizado o comando abaixo:

In [30]:
relevante_str= str(relevante_list)


In [31]:
irrelevante_str= str(irrelevante_list)


Feito isso, agora será possível fazer a limpeza.


In [32]:
relevante_limpo= cleanup(relevante_str)
relevante_limpo

'[rt netflixbrasil assista a filmes e séries por um preço único na netflix e a netflix que caiu 🤡 eu não posso ler as palavras “morte” “assassinato” e afins nessa netflix que eu já fico hmmmmm interessante preciso ver _luisaanogueira saiu da netflix😭😭 mas queria comecei a olhar uma série sobre basquete na netflix  mt boa rt betaadamerdinha vocês pediram a netflix dá \\nbetananetflix teambeta bbtvi  obrigado netflix por mais uma serie de investigação policial pra eu poder perder mais um dia comercial maratonando rt songkangbrasii [infor] | 220321 📸🐼\\n ㅡ veio o primeiro episódio de navillera já está disponível no netflix\\n송강 songkang 나빌레라… vitor_medin tbm penso assim sou viciada nos três netflix me paga por ter tirado 😢 pô netflix lançar a série sem dublagem em português aí vc ferra comigo se ainda fosse inglês o áudio original aí até tudo bem mai nunca que eu vou perdoar a netflix por não ter postos gtnadalt de julie and the phantoms no almanaque tudum netflixbrasil netflix cadê a nov

In [33]:
irrelevante_limpo= cleanup(irrelevante_str)
irrelevante_limpo

'[rt seriesdatvpt maratonas fresquinhas na netflix\\n\\n💃 o thriller espanhol skyrojo \\n🎸 a comédia musical countrycomfort  itsdelicatte esperar a netflix botar uns romance mas sem triângulo pelo amor de deus um enemy to lovers 👁️👄👁️ mysteryguitarm netflix booooa mysteryguitarm  ansioso para ver seu novo trabalho rt limanths tomara que a personagem da giovanna seja do tipo que beija as amigas na balada netflix sirva esse queerbait bem quentinho rt girlgroupsbr seohyun a maknae do snsd será a protagonista do próximo filme da netflix moral sense\\n\\no filme é baseado em webtoon so… milizi misscupece chama fake art e tem no netflix já vi sour grapes arzinho ligado e netflix amo júlia e melina🥺❤\\nfilme socorro virei uma garota na netflix com manu gavassi e thati lopes\\n\\n que mané golpe véi quero que você me chame para assistir netflix chegar lá não ter televisão pompsii amor isso pode banir dua conta da netflix netflix poderia lançar né “cobras em ação” hahahahah ironia \\nsaagas_  m

### Frequência Absoluta

Agora serão contadas as palavras dos tweets e assim construir uma tabela de frequência absoluta para as palavras relevantes, irrelevantes e o conjnto de todas. 

Para isso, vamos converter as strings contendo todos os tweets em objetos do tipo `pd.Series`, nos quais cada elemento é uma palavra. 

<font color='green'> **Relevante**: </font> 

In [34]:
# Quebrando em palavras
relevante_limpo_quebra = relevante_limpo.split()
relevante_limpo_quebra

['[rt',
 'netflixbrasil',
 'assista',
 'a',
 'filmes',
 'e',
 'séries',
 'por',
 'um',
 'preço',
 'único',
 'na',
 'netflix',
 'e',
 'a',
 'netflix',
 'que',
 'caiu',
 '🤡',
 'eu',
 'não',
 'posso',
 'ler',
 'as',
 'palavras',
 '“morte”',
 '“assassinato”',
 'e',
 'afins',
 'nessa',
 'netflix',
 'que',
 'eu',
 'já',
 'fico',
 'hmmmmm',
 'interessante',
 'preciso',
 'ver',
 '_luisaanogueira',
 'saiu',
 'da',
 'netflix😭😭',
 'mas',
 'queria',
 'comecei',
 'a',
 'olhar',
 'uma',
 'série',
 'sobre',
 'basquete',
 'na',
 'netflix',
 'mt',
 'boa',
 'rt',
 'betaadamerdinha',
 'vocês',
 'pediram',
 'a',
 'netflix',
 'dá',
 '\\nbetananetflix',
 'teambeta',
 'bbtvi',
 'obrigado',
 'netflix',
 'por',
 'mais',
 'uma',
 'serie',
 'de',
 'investigação',
 'policial',
 'pra',
 'eu',
 'poder',
 'perder',
 'mais',
 'um',
 'dia',
 'comercial',
 'maratonando',
 'rt',
 'songkangbrasii',
 '[infor]',
 '|',
 '220321',
 '📸🐼\\n',
 'ㅡ',
 'veio',
 'o',
 'primeiro',
 'episódio',
 'de',
 'navillera',
 'já',
 'está',
 

In [35]:
# Guardando as palavras como um pd.Series
relevante_series = pd.Series(relevante_limpo_quebra)
relevante_series

0                 [rt
1       netflixbrasil
2             assista
3                   a
4              filmes
            ...      
2045            tirou
2046           aquela
2047            porra
2048               do
2049        catálogo]
Length: 2050, dtype: object

In [36]:
#Tabela de Frequência absoluta
relevante_absoluta = relevante_series.value_counts()
print(relevante_absoluta)

netflix    128
a           62
de          59
e           46
da          45
          ... 
bruxa        1
chocada      1
veio         1
ctz          1
casal        1
Length: 853, dtype: int64


<font color='violet'>**Irrelevante**: </font>

In [37]:
# Quebrando em palavras
irrelevante_limpo_quebra = irrelevante_limpo.split()
irrelevante_limpo_quebra

['[rt',
 'seriesdatvpt',
 'maratonas',
 'fresquinhas',
 'na',
 'netflix\\n\\n💃',
 'o',
 'thriller',
 'espanhol',
 'skyrojo',
 '\\n🎸',
 'a',
 'comédia',
 'musical',
 'countrycomfort',
 'itsdelicatte',
 'esperar',
 'a',
 'netflix',
 'botar',
 'uns',
 'romance',
 'mas',
 'sem',
 'triângulo',
 'pelo',
 'amor',
 'de',
 'deus',
 'um',
 'enemy',
 'to',
 'lovers',
 '👁️👄👁️',
 'mysteryguitarm',
 'netflix',
 'booooa',
 'mysteryguitarm',
 'ansioso',
 'para',
 'ver',
 'seu',
 'novo',
 'trabalho',
 'rt',
 'limanths',
 'tomara',
 'que',
 'a',
 'personagem',
 'da',
 'giovanna',
 'seja',
 'do',
 'tipo',
 'que',
 'beija',
 'as',
 'amigas',
 'na',
 'balada',
 'netflix',
 'sirva',
 'esse',
 'queerbait',
 'bem',
 'quentinho',
 'rt',
 'girlgroupsbr',
 'seohyun',
 'a',
 'maknae',
 'do',
 'snsd',
 'será',
 'a',
 'protagonista',
 'do',
 'próximo',
 'filme',
 'da',
 'netflix',
 'moral',
 'sense\\n\\no',
 'filme',
 'é',
 'baseado',
 'em',
 'webtoon',
 'so…',
 'milizi',
 'misscupece',
 'chama',
 'fake',
 'art',
 

In [38]:
# Guardando as palavras como um pd.Series
irrelevante_series = pd.Series(irrelevante_limpo_quebra)
irrelevante_series

0                [rt
1       seriesdatvpt
2          maratonas
3        fresquinhas
4                 na
            ...     
3325              pc
3326          melhor
3327             que
3328              no
3329     celular\n\n
Length: 3330, dtype: object

In [39]:
#Tabela de Frequência absoluta
irrelevante_absoluta = irrelevante_series.value_counts()
print(irrelevante_absoluta)

netflix         148
que              82
a                80
e                78
de               76
               ... 
arregimento       1
ʚ                 1
namorando         1
sim\n12           1
promocionais      1
Length: 1388, dtype: int64


<font color='purple'> **Todas (relevantes e irrelevantes)**: </font> 

In [40]:
geral = irrelevante_limpo + relevante_limpo
geral_serie = pd.Series(geral.split())
geral_absoluta= geral_serie.value_counts() 
print(geral_absoluta)

netflix            276
a                  142
de                 135
que                126
e                  124
                  ... 
tentando             1
bloquead             1
mariada12060561      1
ch…                  1
jovemendex           1
Length: 1919, dtype: int64


### Probabilidades que devemos calcular a partir da base de dados:

Serão obtidos tais probabilidades nos seguintes passos:

 1. 
  A) Calcular a probabilide $P(palavra|Relevante)$, usando o comando `value_counts(True)`.

In [41]:
relevante_relativa = relevante_series.value_counts(True)*100
print(relevante_relativa)

netflix    6.243902
a          3.024390
de         2.878049
e          2.243902
da         2.195122
             ...   
bruxa      0.048780
chocada    0.048780
veio       0.048780
ctz        0.048780
casal      0.048780
Length: 853, dtype: float64


B) Calcular a probabilide $P(palavra|Irrelevante)$, usando o comando `value_counts(True)`.

In [42]:
irrelevante_relativa = irrelevante_series.value_counts(True)*100
print(irrelevante_relativa)

netflix         4.444444
que             2.462462
a               2.402402
e               2.342342
de              2.282282
                  ...   
arregimento     0.030030
ʚ               0.030030
namorando       0.030030
sim\n12         0.030030
promocionais    0.030030
Length: 1388, dtype: float64


 2. Probabilidade de ser Relevante ou Irrelevante: calcular $𝑃(R)$ e $𝑃(I)$ a partir da base da base de dados na planilha "Treinamento". Usando o comando `value_counts(True)`.
  


* 𝑃 (R)  

In [43]:
prob_relevante= treinamento.B1.value_counts(True)[1]
prob_relevante

0.43333333333333335

* 𝑃(𝐼)  

In [44]:
prob_irrelevante= 1- prob_relevante
prob_irrelevante

0.5666666666666667

## Classificador

O cálculo da probabilidade será realizado utilizando o Teorema de Bayes:
<img src="Teorema de Bayes.png">


In [45]:
#Puxando a frase
for frase in teste.Teste:
    frase_usada= cleanup(frase.lower())
    frase_usada= frase_usada.split()
    
    irrelevante_final = 1
    relevante_final= 1
    
#Montando as probabilidades 
    
    for i in frase_usada:
        if i in irrelevante_absoluta:
            irrelevante_final*= (irrelevante_absoluta[i]+1)/(len(irrelevante_series) + len(geral_absoluta))
        if i in relevante_absoluta:
            relevante_final*= (relevante_absoluta[i]+1)/(len(relevante_series) + len(geral_absoluta) )
        
        if i not in irrelevante_absoluta:
            irrelevante_final*= 1/(len(irrelevante_series) + len(geral_absoluta))
            
        if i not in relevante_absoluta:
            relevante_final*= 1/(len(relevante_series) + len(geral_absoluta))
    
            
#Determinando como vai aparecer a classificação
    
    
    if (relevante_final*prob_relevante > irrelevante_final* prob_irrelevante):
        teste.loc[(teste.Teste==frase),'Relevância'] = 'Relevante'
    else:
        teste.loc[(teste.Teste==frase),'Relevância'] = 'Irrelevante'
        
teste.head(15)
    

,Teste,B1,Relevância
0,@pdrojao amigoooooo tô precisando de uma série...,0,Relevante
1,"acabei cidade invisível, achei que a netflix i...",1,Relevante
2,"pra quem gosta de comédia, assistam ""cabra da ...",1,Relevante
3,@tyongenrose @iucasmoon helpp okay well comece...,0,Relevante
4,"a mina simplesmente senta, pega um punhal e di...",0,Relevante
5,"@gabymonks16 acho que na netflix não, gaby. eu...",0,Irrelevante
6,"eu aprendi a mexer na netflix party pelo pc, a...",0,Irrelevante
7,rt @galeradamanu: sabemos que nossa manoela es...,1,Irrelevante
8,queria assistir cobra kai sem precisar de netflix,1,Relevante
9,@netflixbrasil @netflix esperando a renovação ...,1,Relevante


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [46]:
pd.crosstab(teste.Relevância, teste.B1, normalize=True)*100

B1,0,1
Relevância,,
Irrelevante,22.0,14.0
Relevante,28.5,35.5


___
### Concluindo

A partir dos dados coletados na verificação de performance do classificador é possível concluir que foi obtida uma acurácia total de 57,5%, dos quais 35,5% são verdadeiros positivos, 28,5% falsos positivos, 22% verdadeiros negativos e 14% falsos negativos, o que é considerado um reusltado mediano.

Isso ocorre devido a dificuldade na classificações de tweets manualmente, o que reflete no desempenho do classificador.
Um fator importante que dificulta o processo são as figuras de linguagem como sarcasmo e ironia. Elas são formas de expressar uma afirmação com sentido conotativo, ou seja, figurativo. A ironia, por exemplo, é uma inversão do real significado de algo em determinado contexto quando se diz o contrário do que se pensa, como em "Nossa, você é mesmo muito gentil". Nesse exemplo, se não levar em cosideração o contexto pode ser interpretado como um elogio.

Tanto na classificação manual quanto pelo classificador não é levado em consideração esse aspecto por não sabermos o contexto no qual o tweet foi publicado.

O que ajuda também a distinguir para a classificação manual são os emojis, como por exemplo o de paplhaço após a frase. Para uma pessoa olhando fica clara a interpertação porém o classificador não leva em consideração os emojis.

#### Plano de expansão

O cliente deve continuar financiando o projeto pois traz informações importantes para a melhoria da empresa. Com um maior capital será possível viabilizar melhorias no classificador como aumentar a base de dados para treinamento com atualizações períodicas, tornando o classificador mais preciso já que nosso produto é utilizado no Brasil inteiro e por várias faixas etárias e assim temos uma grande abrangência linguística. 

Aplicar limpezas mais avançadas como substituição de palavras gramaticalmente erradas e interpretação de emojis também seriam fatores que melhorariam muito a acurácia do classificador.

#### Podemos alimentar a base de treinamento com o próprio classificador?

Não se deve alimentar a base de treinamento com o próprio classificador devido ao fato dos erros já existentes serem propagados e, consequentemente, gerarem piores resultados, causando uma maior imprecisão nos dados e uma menor confiabilidade do modelo.

#### Outras funcionalidades para o classificador

É possível observar outras funcionalidades para o classificador como analisar cenários políticos durante eleições, probabilidade de contrair uma doença e se curar dados os resultados anteriores. Outra opção seria a probabilidade de uma frase ser certo estilo musical ou artista. 

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências


[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**